# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p sc:name ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-e1a4ae61c4-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Book Workflow Series ("Nobel laureates explorative search") 


Consider the following exploratory scenario:

>  Investigate the authors who won the Nobel award for literature and get information about the nationality of the winners and analyse their literary production



## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wd:Q37922`   | Nobel Prize in Literature     | node |
| `wd:Q36180`   | writer        | node |
| `wd:Q38`      | Italy         | node |
| `wd:Q3624078` | Sovereign state  | node |
| `wd:Q213678`  | Vatican Library  | node |


Also consider

```
?p wdt:P27 wd:Q38  . 
?p wdt:P106/wdt:P279* wd:Q36180  . 
```

is the BGP to retrieve all **italian authors**

The workload should


1. Identify the BGP for obtaining the nobel laureates and the nominees with their relevant attributes

2. Get the number of Italian, French and Iranian winners and check which nation won more awards in the last twenty years.

3. Determine how many Literature nobel laureates have the following contries: Italy, Germany, France, Romania, Denmark, Iran, and China

4. How many Literature Nobel award winners have a PhD (aka Doctorate of Philosophy) (you may check if they have a doctoral advisor)?

5. Are there books from Litarature Nobel Award winners which are not present in the Vatican Library? (if so, who is the author with more books not in the Vatical Library)?

In [2]:
# start your workflow here

In [3]:
queryString = """
SELECT COUNT( ?p)
WHERE { 
?p wdt:P27 wd:Q38  . 
?p wdt:P106/wdt:P279* wd:Q36180  . 

} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '46967')]


1

## Task 1
Identify the BGP for obtaining the nobel laureates and the nominees with their relevant attributes

To answer that question I will start from `wd:Q37922` that is the node Nobel Prize in literature and look for its superclass, hoping that I will find he node 'Nobel Prize'

In [4]:
queryString = """
SELECT DISTINCT ?super ?name
WHERE {  
wd:Q37922 wdt:P279* ?super  . 
?super sc:name ?name .
} 
"""

print("Results")
run_query(queryString)

Results
[('super', 'http://www.wikidata.org/entity/Q7302601'), ('name', 'recognition')]
[('super', 'http://www.wikidata.org/entity/Q7191'), ('name', 'Nobel Prize')]
[('super', 'http://www.wikidata.org/entity/Q937228'), ('name', 'property')]
[('super', 'http://www.wikidata.org/entity/Q618779'), ('name', 'award')]
[('super', 'http://www.wikidata.org/entity/Q35120'), ('name', 'entity')]
[('super', 'http://www.wikidata.org/entity/Q189970'), ('name', 'social status')]
[('super', 'http://www.wikidata.org/entity/Q11424100'), ('name', 'status')]
[('super', 'http://www.wikidata.org/entity/Q11028'), ('name', 'information')]
[('super', 'http://www.wikidata.org/entity/Q1190554'), ('name', 'occurrence')]
[('super', 'http://www.wikidata.org/entity/Q37922'), ('name', 'Nobel Prize in Literature')]
[('super', 'http://www.wikidata.org/entity/Q1207505'), ('name', 'quality')]
[('super', 'http://www.wikidata.org/entity/Q7184903'), ('name', 'abstract object')]
[('super', 'http://www.wikidata.org/entity/Q489

21

The interest node is `wd:Q7191` that is the Nobel Prize. Going on with the notebook I will use the triple `?p wdt:P31/wdt:P279* ?wd:Q7191` to identify all the nobel prize. Now look for some example of subclass of Nobel Prize (`wd:Q7191`)

In [5]:
queryString = """
SELECT DISTINCT ?p ?name
WHERE {  
?p wdt:P31/wdt:P279* wd:Q7191  . 
?p sc:name ?name .
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q5959408'), ('name', '2003 Nobel Peace Prize')]
[('p', 'http://www.wikidata.org/entity/Q10844039'), ('name', '1989 Nobel Peace Prize')]
[('p', 'http://www.wikidata.org/entity/Q21078165'), ('name', '2015 Nobel Peace Prize')]
[('p', 'http://www.wikidata.org/entity/Q27132539'), ('name', '2016 Nobel Peace Prize')]
[('p', 'http://www.wikidata.org/entity/Q42417649'), ('name', '2017 Nobel Peace Prize')]
[('p', 'http://www.wikidata.org/entity/Q59653078'), ('name', '2018 Nobel Peace Prize')]
[('p', 'http://www.wikidata.org/entity/Q48912926'), ('name', 'Nobel Prize in Physics 2015')]
[('p', 'http://www.wikidata.org/entity/Q48913298'), ('name', 'Nobel Prize in Chemistry 2017')]
[('p', 'http://www.wikidata.org/entity/Q21187755'), ('name', '2004 Nobel Peace Prize')]
[('p', 'http://www.wikidata.org/entity/Q4604682'), ('name', '2005 Nobel Peace Prize')]
[('p', 'http://www.wikidata.org/entity/Q48913291'), ('name', 'Nobel Prize in Chemistry 2016')]
[('p', 

39

Only 39 Nobel Prize at all seems not too much. Looking for some properties of Nobel Prize

In [6]:
queryString = """
SELECT DISTINCT ?pr ?name
WHERE {  
wd:Q7191 ?pr ?o . 
?pr sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1051'), ('name', 'PSH ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P112'), ('name', 'founded by')]
[('pr', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('pr', 'http://www.wikidata.org/prop/direct/P1225'), ('name', 'U.S. National Archives Identifier')]
[('pr', 'http://www.wikidata.org/prop/direct/P1245'), ('name', 'OmegaWiki Defined Meaning')]
[('pr', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('pr', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('pr', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('pr', 'http://www.wikidata.org/prop/direct/P1438'), ('name', 'Jewish Encyclopedia ID (Russian)')

72

In [7]:
#looking at the property hasPart wdt:P527
queryString = """
SELECT DISTINCT ?o ?name
WHERE {  
wd:Q7191 wdt:P527 ?o . 
?o sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q35637'), ('name', 'Nobel Peace Prize')]
[('o', 'http://www.wikidata.org/entity/Q37922'), ('name', 'Nobel Prize in Literature')]
[('o', 'http://www.wikidata.org/entity/Q38104'), ('name', 'Nobel Prize in Physics')]
[('o', 'http://www.wikidata.org/entity/Q44585'), ('name', 'Nobel Prize in Chemistry')]
[('o', 'http://www.wikidata.org/entity/Q80061'), ('name', 'Nobel Prize in Physiology or Medicine')]


5

In this way I found all the type of nobel prizes exist in the database. Verify if the result of 39 nobel prizes found above is the same if I look for objects that is instance of this nodes that I've just found. 

In [8]:
queryString = """
SELECT DISTINCT ?sub ?name
WHERE {  
wd:Q7191 wdt:P527 ?o . 
?sub wdt:P31 ?o.
?sub sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q5959408'), ('name', '2003 Nobel Peace Prize')]
[('sub', 'http://www.wikidata.org/entity/Q10844039'), ('name', '1989 Nobel Peace Prize')]
[('sub', 'http://www.wikidata.org/entity/Q21078165'), ('name', '2015 Nobel Peace Prize')]
[('sub', 'http://www.wikidata.org/entity/Q27132539'), ('name', '2016 Nobel Peace Prize')]
[('sub', 'http://www.wikidata.org/entity/Q42417649'), ('name', '2017 Nobel Peace Prize')]
[('sub', 'http://www.wikidata.org/entity/Q59653078'), ('name', '2018 Nobel Peace Prize')]
[('sub', 'http://www.wikidata.org/entity/Q48912926'), ('name', 'Nobel Prize in Physics 2015')]
[('sub', 'http://www.wikidata.org/entity/Q48913298'), ('name', 'Nobel Prize in Chemistry 2017')]
[('sub', 'http://www.wikidata.org/entity/Q21187755'), ('name', '2004 Nobel Peace Prize')]
[('sub', 'http://www.wikidata.org/entity/Q4604682'), ('name', '2005 Nobel Peace Prize')]
[('sub', 'http://www.wikidata.org/entity/Q48913291'), ('name', 'Nobel Prize in Che

39

In [9]:
#looking at the subclasses of nobel prizes
queryString = """
SELECT DISTINCT ?sub ?name
WHERE {  
?sub wdt:P279+ wd:Q7191 . 
?sub sc:name ?name .
} 
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q35637'), ('name', 'Nobel Peace Prize')]
[('sub', 'http://www.wikidata.org/entity/Q37922'), ('name', 'Nobel Prize in Literature')]
[('sub', 'http://www.wikidata.org/entity/Q38104'), ('name', 'Nobel Prize in Physics')]
[('sub', 'http://www.wikidata.org/entity/Q44585'), ('name', 'Nobel Prize in Chemistry')]
[('sub', 'http://www.wikidata.org/entity/Q47170'), ('name', 'Prize in Economic Sciences in Memory of Alfred Nobel')]
[('sub', 'http://www.wikidata.org/entity/Q80061'), ('name', 'Nobel Prize in Physiology or Medicine')]


6

In this way I have also the economic science's nobel price. But from the information found some queries ago, it seems that I have no entity that is istance of nobel prize in economic science. Run an aggregation query to found some more information. 

In [10]:
#group by subclass of Nobel Prize
queryString = """
SELECT DISTINCT ?name (COUNT (?p) AS ?count)
WHERE {  
?sub wdt:P279+ wd:Q7191 . 
?p wdt:P31 ?sub.
?sub sc:name ?name .
} 
GROUP BY ?sub ?name
ORDER BY DESC (?count) 
"""

print("Results")
run_query(queryString)

Results
[('name', 'Nobel Peace Prize'), ('count', '21')]
[('name', 'Nobel Prize in Physiology or Medicine'), ('count', '7')]
[('name', 'Nobel Prize in Physics'), ('count', '7')]
[('name', 'Nobel Prize in Chemistry'), ('count', '6')]
[('name', 'Nobel Prize in Literature'), ('count', '2')]


5

In [11]:
#RE-run the query with the optional, since the economic sciences does not match anything
queryString = """
SELECT DISTINCT ?name (COUNT (?p) AS ?count)
WHERE {  
?sub wdt:P279+ wd:Q7191 . 
OPTIONAL {?p wdt:P31 ?sub.}
?sub sc:name ?name .
} 
GROUP BY ?sub ?name
ORDER BY DESC (?count) 
"""

print("Results")
run_query(queryString)

Results
[('name', 'Nobel Peace Prize'), ('count', '21')]
[('name', 'Nobel Prize in Physiology or Medicine'), ('count', '7')]
[('name', 'Nobel Prize in Physics'), ('count', '7')]
[('name', 'Nobel Prize in Chemistry'), ('count', '6')]
[('name', 'Nobel Prize in Literature'), ('count', '2')]
[('name', 'Prize in Economic Sciences in Memory of Alfred Nobel'), ('count', '0')]


6

That's conclude the first analysis about Nobel Prizes, types and how they are distributed. Now look for nominees.

In [12]:
#search in direction NOBEL TO OBJECT
queryString = """
SELECT DISTINCT ?pr ?name
WHERE {  
?nobel wdt:P31/wdt:P279* wd:Q7191 ;
        ?pr ?o.

?pr sc:name ?name .
}
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1269'), ('name', 'facet of')]
[('pr', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('pr', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('pr', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('pr', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('pr', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('pr', 'http://www.wikidata.org/prop/direct/P179'), ('name', 'part of the series')]
[('pr', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('pr', 'http://www.wikidata.org/prop/direct/P2671'), ('name', 'Google Knowledge Graph ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('pr', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('pr', 'http://www.wikidata.org/prop/direct/P3219'), ('name', 'Encyclopædia Universalis ID')]
[('pr', 'http://www.wikidata.org

19

In [13]:
#search in direction OBJECT TO NOBEL
queryString = """
SELECT DISTINCT ?pr ?name
WHERE {  
?nobel wdt:P31/wdt:P279* wd:Q7191 .
?o ?pr ?nobel.

?pr sc:name ?name .
}
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1423'), ('name', 'template has topic')]
[('pr', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('pr', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('pr', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('pr', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('pr', 'http://www.wikidata.org/prop/direct/P921'), ('name', 'main subject')]


6

Do a test with one Nobel Prize to understand and find a path from people that is nominated / win a nobel prize, because I found the interest properties:
- `wdt:P1346`: winner (to use `?nobel wdt:P1346 ?people`);
- `wdt:P1411`: nominated for(to use `?people wdt:P1346 ?nobel`).

In [14]:
queryString = """
SELECT DISTINCT ?nNobel ?nPeople
WHERE {  
?nobel wdt:P31/wdt:P279* wd:Q7191 .
?people wdt:P1411 ?nobel.
?nobel sc:name ?nNobel .
?people sc:name ?nPeople .

OPTIONAL{?nobel wdt:P1346 ?people.}
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('nNobel', '2010 Nobel Peace Prize'), ('nPeople', 'Tobias Asser')]


1

Very poor result. Maybe reification, so start looking for an entity that is connected to both nobel and people (I assume that I have to look for people that is nominated for a nobel prize, but anyway this constraint is not needed).

In [15]:
queryString = """
SELECT DISTINCT ?o ?unknown
WHERE {  
?nobel wdt:P31/wdt:P279+ wd:Q7191 .
?o ?pr1 ?nobel.
?x ?pr2 ?o.
?o sc:name ?unknown .

}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('o', 'http://www.wikidata.org/entity/Q5959408'), ('unknown', '2003 Nobel Peace Prize')]
[('o', 'http://www.wikidata.org/entity/Q216872'), ('unknown', 'Tobias Asser')]
[('o', 'http://www.wikidata.org/entity/Q21078165'), ('unknown', '2015 Nobel Peace Prize')]
[('o', 'http://www.wikidata.org/entity/Q27132539'), ('unknown', '2016 Nobel Peace Prize')]
[('o', 'http://www.wikidata.org/entity/Q4618031'), ('unknown', '2010 Nobel Peace Prize')]
[('o', 'http://www.wikidata.org/entity/Q3231462'), ('unknown', '2009 Nobel Peace Prize')]
[('o', 'http://www.wikidata.org/entity/Q7045636'), ('unknown', '2013 Nobel Peace Prize')]
[('o', 'http://www.wikidata.org/entity/Q42417649'), ('unknown', '2017 Nobel Peace Prize')]
[('o', 'http://www.wikidata.org/entity/Q59653078'), ('unknown', '2018 Nobel Peace Prize')]
[('o', 'http://www.wikidata.org/entity/Q21187755'), ('unknown', '2004 Nobel Peace Prize')]
[('o', 'http://www.wikidata.org/entity/Q4604682'), ('unknown', '2005 Nobel Peace Prize')]
[('o', '

22

In [16]:
#look at the connection property
queryString = """
SELECT DISTINCT ?pr1 ?unknown ?name
WHERE {  
?nobel wdt:P31/wdt:P279+ wd:Q7191 .
?o ?pr1 ?nobel.
?x ?pr2 ?o.
?pr1 sc:name ?unknown .
?nobel sc:name ?name .

}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr1', 'http://www.wikidata.org/prop/direct/P155'), ('unknown', 'follows'), ('name', '2002 Nobel Peace Prize')]
[('pr1', 'http://www.wikidata.org/prop/direct/P156'), ('unknown', 'followed by'), ('name', '2004 Nobel Peace Prize')]
[('pr1', 'http://www.wikidata.org/prop/direct/P1411'), ('unknown', 'nominated for'), ('name', '2010 Nobel Peace Prize')]
[('pr1', 'http://www.wikidata.org/prop/direct/P155'), ('unknown', 'follows'), ('name', '2014 Nobel Peace Prize')]
[('pr1', 'http://www.wikidata.org/prop/direct/P156'), ('unknown', 'followed by'), ('name', '2016 Nobel Peace Prize')]
[('pr1', 'http://www.wikidata.org/prop/direct/P155'), ('unknown', 'follows'), ('name', '2015 Nobel Peace Prize')]
[('pr1', 'http://www.wikidata.org/prop/direct/P156'), ('unknown', 'followed by'), ('name', '2017 Nobel Peace Prize')]
[('pr1', 'http://www.wikidata.org/prop/direct/P156'), ('unknown', 'followed by'), ('name', '2011 Nobel Peace Prize')]
[('pr1', 'http://www.wikidata.org/prop/direct/P155'), ('u

40

This is not what I need. So I will search for property to the generical type of nobel prize, not the specific year

In [17]:
queryString = """
SELECT DISTINCT ?pr ?name 
WHERE {  
?sub wdt:P279+ wd:Q7191 . 
?p ?pr ?sub.
?pr sc:name ?name .
}  
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P3712'), ('name', 'objective of project or action')]
[('pr', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]
[('pr', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('pr', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('pr', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('pr', 'http://www.wikidata.org/prop/direct/P179'), ('name', 'part of the series')]
[('pr', 'http://www.wikidata.org/prop/direct/P1889'), ('name', 'different from')]
[('pr', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('pr', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('pr', 'http://www.wikidata.org/prop/direct/P360'), ('name', 'is a list of')]
[('pr', 'http://www.wikidata.org/prop/direct/P366'), ('name', 'use')]
[('pr', 'http://www.wikidata.org/prop/direct/P463'), ('name', 'member of')]
[('pr', 'http://w

19

Found two interest property that are:
- `wdt:P166` : award received;
- `wdt:P1411` : nominated for.

In [18]:
#Visualize some people that receive the nobel prize
queryString = """
SELECT DISTINCT ?people ?nobel 
WHERE {  
?sub wdt:P279+ wd:Q7191 . 
?p wdt:P166 ?sub.
?p sc:name ?people .
?sub sc:name ?nobel .
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('people', 'Daniel Bovet'), ('nobel', 'Nobel Prize in Physiology or Medicine')]
[('people', 'Barry C. Barish'), ('nobel', 'Nobel Prize in Physics')]
[('people', 'Alan J. Heeger'), ('nobel', 'Nobel Prize in Chemistry')]
[('people', 'Gerd Binnig'), ('nobel', 'Nobel Prize in Physics')]
[('people', 'Robert Coleman Richardson'), ('nobel', 'Nobel Prize in Physics')]
[('people', 'Frank B. Kellogg'), ('nobel', 'Nobel Peace Prize')]
[('people', 'Harry Markowitz'), ('nobel', 'Prize in Economic Sciences in Memory of Alfred Nobel')]
[('people', 'Otto Stern'), ('nobel', 'Nobel Prize in Physics')]
[('people', 'Klaus von Klitzing'), ('nobel', 'Nobel Prize in Physics')]
[('people', 'Tawakel Karman'), ('nobel', 'Nobel Peace Prize')]
[('people', 'Johannes V. Jensen'), ('nobel', 'Nobel Prize in Literature')]
[('people', 'Luc Montagnier'), ('nobel', 'Nobel Prize in Physiology or Medicine')]
[('people', 'Françoise Barré-Sinoussi'), ('nobel', 'Nobel Prize in Physiology or Medicine')]
[('people', 'J

20

In [19]:
#Visualize some people that was nominated for a nobel prize
queryString = """
SELECT DISTINCT ?people ?nobel 
WHERE {  
?sub wdt:P279+ wd:Q7191 . 
?p wdt:P1411 ?sub.
?p sc:name ?people .
?sub sc:name ?nobel .
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('people', 'Valery Larbaud'), ('nobel', 'Nobel Prize in Literature')]
[('people', 'Edward Grey, 1st Viscount Grey of Fallodon'), ('nobel', 'Nobel Peace Prize')]
[('people', 'Daniel Bovet'), ('nobel', 'Nobel Prize in Physiology or Medicine')]
[('people', 'Howard Cary'), ('nobel', 'Nobel Prize in Chemistry')]
[('people', 'Adrien Albert Marie de Mun'), ('nobel', 'Nobel Prize in Literature')]
[('people', 'Alan Sillitoe'), ('nobel', 'Nobel Prize in Literature')]
[('people', 'Pierre Emmanuel'), ('nobel', 'Nobel Prize in Literature')]
[('people', 'Ramón Pérez de Ayala'), ('nobel', 'Nobel Prize in Literature')]
[('people', 'Franz Theodor Csokor'), ('nobel', 'Nobel Prize in Literature')]
[('people', 'Frank B. Kellogg'), ('nobel', 'Nobel Peace Prize')]
[('people', 'Otto Stern'), ('nobel', 'Nobel Prize in Physics')]
[('people', 'Rudolf Alexander Schröder'), ('nobel', 'Nobel Prize in Literature')]
[('people', 'Giorgos Theotokás'), ('nobel', 'Nobel Prize in Literature')]
[('people', 'Johan

20

Visualize the total number of people nominated for a nobel prize and the total number of people that won a nobel prize

In [20]:
queryString = """
SELECT ?tot_nominated ?tot_winner
WHERE {  
    {
        SELECT (COUNT(*)) AS ?tot_nominated 
        WHERE{ 
            ?sub wdt:P279+ wd:Q7191 . 
            ?p wdt:P1411 ?sub.
        }
    }
    {
        SELECT (COUNT(*)) AS ?tot_winner
        WHERE{ 
            ?sub wdt:P279+ wd:Q7191 . 
            ?p wdt:P166 ?sub.
        }
    }
}
"""

print("Results")
run_query(queryString)

Results
[('tot_nominated', '2576'), ('tot_winner', '960')]


1

Since the result, before count all the matches in the database, I want to know if there are people nominated twice or more and maybe someone won a nobel prize twice, so count DISTINCT people

In [21]:
queryString = """
SELECT ?tot_nominated ?tot_winner
WHERE {  
    {
        SELECT (COUNT(DISTINCT ?p)) AS ?tot_nominated 
        WHERE{ 
            ?sub wdt:P279+ wd:Q7191 . 
            ?p wdt:P1411 ?sub.
        }
    }
    {
        SELECT (COUNT(DISTINCT ?p)) AS ?tot_winner
        WHERE{ 
            ?sub wdt:P279+ wd:Q7191 . 
            ?p wdt:P166 ?sub.
        }
    }
}
"""

print("Results")
run_query(queryString)

Results
[('tot_nominated', '2395'), ('tot_winner', '958')]


1

The result shows that actually there are 2 people that won a nobel prize twice and a lot of people nominated more than once. Only for curiosity, show who won more than one Nobel Prize (I know that Marie Curie won two nobel prize so I expect to find her)

In [22]:
queryString = """
SELECT DISTINCT ?people (GROUP_CONCAT( ?nobel;SEPARATOR=" ; ") AS ?prizes)
WHERE{ 
    ?s wdt:P279+ wd:Q7191 . 
    ?pI wdt:P166 ?s.
    ?pI sc:name ?people .
    ?s sc:name ?nobel .
}
GROUP BY ?pI ?people
HAVING ((COUNT(?nobel)) > 1)
"""

print("Results")
run_query(queryString)

Results
[('people', 'Linus Pauling'), ('prizes', 'Nobel Peace Prize ; Nobel Prize in Chemistry')]
[('people', 'Marie Curie'), ('prizes', 'Nobel Prize in Physics ; Nobel Prize in Chemistry')]


2

Only for curiosity, I will perform an ask query to know if exists people that are nominated more than twice for a Nobel Prize

In [23]:
queryString = """
ASK{
    SELECT DISTINCT ?p{ 
        ?s wdt:P279+ wd:Q7191 . 
        ?p wdt:P1411 ?s.
    }
    GROUP BY ?p
    HAVING ((COUNT(?s)) > 3)
}
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': False}

## Task 2
Get the number of Italian, French and Iranian winners and check which nation won more awards in the last twenty years.

To answer that question I need to find the nodes Italy, France and Iran, that will allow me to verify the citizenship of a people and count the winners. Starting from one of the entry point provided by the workflow, `wd:Q38` that refers to the  country Italy, I will find the countries in one shot.

In [24]:
queryString = """
SELECT DISTINCT ?c ?country
WHERE{ 
    wd:Q38 wdt:P31 ?o . 
    ?c wdt:P31 ?o.
    ?c sc:name ?country .
    ?o sc:name ?obj .
    FILTER(REGEX(?country, "[I,i]taly") OR REGEX(?country, "[F,f]rance") OR REGEX(?country, "[I,i]ran") )
    
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q794'), ('country', 'Iran')]
[('c', 'http://www.wikidata.org/entity/Q107258515'), ('country', 'Pahlavi Iran')]
[('c', 'http://www.wikidata.org/entity/Q142'), ('country', 'France')]
[('c', 'http://www.wikidata.org/entity/Q38'), ('country', 'Italy')]


4

The country needed are:
- `wdt:Q38` Italy;
- `wdt:Q142`  France;
- `wdt:Q794`  Iran.

Now I can answer of the first part of the question: count the number of Nobel Prize's winners of these country, but to perform quicly the group by I will include the query above in the query below

In [25]:
queryString = """
SELECT DISTINCT ?country (COUNT(*) AS ?count)
WHERE {  
?sub wdt:P279+ wd:Q7191 . 
?p wdt:P166 ?sub;
    wdt:P27 ?c.
#part related to the country
wd:Q38 wdt:P31 ?o . 
?c wdt:P31 ?o.
?c sc:name ?country .
FILTER(REGEX(?country, "[I,i]taly") OR REGEX(?country, "[F,f]rance") OR REGEX(?country, "[I,i]ran") )
    
}
GROUP BY ?c ?country
ORDER BY DESC(?count) 
"""

print("Results")
run_query(queryString)

Results
[('country', 'France'), ('count', '142')]
[('country', 'Italy'), ('count', '26')]
[('country', 'Iran'), ('count', '2')]


3

To answer the second part of the question I need to find something about date, and when a people won the Award. From one of the first query I found that there exist a specific object for every (nobel,year), so I assume that something similar exists also for the generic award. Starting from that I will explore this (reminder: **Q618779** is the award).

In [26]:
#example of these objects
queryString = """
SELECT DISTINCT ?sub ?name
WHERE {  
?sub wdt:P31/wdt:P279* wd:Q618779 .

    
?sub sc:name ?name .
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q59999031'), ('name', 'Patriarch of Eritrea')]
[('sub', 'http://www.wikidata.org/entity/Q16205759'), ('name', 'Bishop of Ross')]
[('sub', 'http://www.wikidata.org/entity/Q13055631'), ('name', "People's Writer of Bashkortostan")]
[('sub', 'http://www.wikidata.org/entity/Q6888520'), ('name', 'Moderator of the General Assembly')]
[('sub', 'http://www.wikidata.org/entity/Q1788732'), ('name', 'Bulgarian Commemorative Medal for Balkan Wars')]
[('sub', 'http://www.wikidata.org/entity/Q435017'), ('name', 'emeritus bishop')]
[('sub', 'http://www.wikidata.org/entity/Q4768306'), ('name', 'Anne Elder Award')]
[('sub', 'http://www.wikidata.org/entity/Q2011427'), ('name', 'Order of Bishop Platon')]
[('sub', 'http://www.wikidata.org/entity/Q4828639'), ('name', 'Aviation Hall of Fame and Museum of New Jersey')]
[('sub', 'http://www.wikidata.org/entity/Q1235107'), ('name', 'Finnish Hockey Hall of Fame')]


10

In [27]:
#just for curiosity verify how many different award exists
queryString = """
SELECT COUNT(DISTINCT ?sub) AS ?awards
WHERE {  
?sub wdt:P279+ wd:Q618779 .
} 
"""

print("Results")
run_query(queryString)

Results
[('awards', '7038')]


1

In [28]:
queryString = """
SELECT DISTINCT ?sp ?name
WHERE {  
?sub wdt:P279+ wd:Q618779 .
?sp wdt:P31 ?sub.

?sp sc:name ?name .
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('sp', 'http://www.wikidata.org/entity/Q59999031'), ('name', 'Patriarch of Eritrea')]
[('sp', 'http://www.wikidata.org/entity/Q16205759'), ('name', 'Bishop of Ross')]
[('sp', 'http://www.wikidata.org/entity/Q13055631'), ('name', "People's Writer of Bashkortostan")]
[('sp', 'http://www.wikidata.org/entity/Q6888520'), ('name', 'Moderator of the General Assembly')]
[('sp', 'http://www.wikidata.org/entity/Q1788732'), ('name', 'Bulgarian Commemorative Medal for Balkan Wars')]
[('sp', 'http://www.wikidata.org/entity/Q435017'), ('name', 'emeritus bishop')]
[('sp', 'http://www.wikidata.org/entity/Q4768306'), ('name', 'Anne Elder Award')]
[('sp', 'http://www.wikidata.org/entity/Q2011427'), ('name', 'Order of Bishop Platon')]
[('sp', 'http://www.wikidata.org/entity/Q4828639'), ('name', 'Aviation Hall of Fame and Museum of New Jersey')]
[('sp', 'http://www.wikidata.org/entity/Q1235107'), ('name', 'Finnish Hockey Hall of Fame')]
[('sp', 'http://www.wikidata.org/entity/Q2046452'), ('name',

20

In [29]:
#look for property of these objects
queryString = """
SELECT DISTINCT ?pr ?name
WHERE {  
?sub wdt:P279+ wd:Q618779 .
?sp wdt:P31 ?sub;
    ?pr ?o.

    
?pr sc:name ?name .
FILTER(REGEX(?name,"[T,t]ime") OR REGEX(?name,"[W,w]in") OR REGEX(?name,"[R,r]eceive") )
} 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('pr', 'http://www.wikidata.org/prop/direct/P580'), ('name', 'start time')]
[('pr', 'http://www.wikidata.org/prop/direct/P585'), ('name', 'point in time')]
[('pr', 'http://www.wikidata.org/prop/direct/P166'), ('name', 'award received')]
[('pr', 'http://www.wikidata.org/prop/direct/P2348'), ('name', 'time period')]
[('pr', 'http://www.wikidata.org/prop/direct/P3221'), ('name', 'New York Times topic ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P582'), ('name', 'end time')]
[('pr', 'http://www.wikidata.org/prop/direct/P421'), ('name', 'located in time zone')]
[('pr', 'http://www.wikidata.org/prop/direct/P190'), ('name', 'twinned administrative body')]
[('pr', 'http://www.wikidata.org/prop/direct/P1249'), ('name', 'time of earliest written record')]
[('pr', 'http://www.wikidata.org/prop/direct/P5719'), ('name', 'National Medal of Arts winner ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P7768'), (

15

The interesting predicate are:
- winner (`wdt:P1346`);
- point in time (`wdt:P585`).

In [30]:
queryString = """
SELECT DISTINCT ?year ?award ?name
WHERE {  
?sub wdt:P279+ wd:Q618779 .
?sp wdt:P31 ?sub;
    wdt:P585 ?o;
    wdt:P1346 ?win.

BIND(year(xsd:dateTime(?o)) AS ?year)

?sp sc:name ?award .
?win sc:name ?name .
} 
ORDER BY ?year
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('year', '1982'), ('award', 'Xabier Lizardi award'), ('name', 'minus sign')]
[('year', '1982'), ('award', 'Xabier Lizardi award'), ('name', 'Arrate Egaña')]
[('year', '1982'), ('award', 'Xabier Lizardi award'), ('name', 'Xabier Mendiguren Elizegi')]
[('year', '1982'), ('award', 'Xabier Lizardi award'), ('name', 'Castillo Suárez')]
[('year', '1982'), ('award', 'Xabier Lizardi award'), ('name', 'Ruben Ruiz')]
[('year', '1982'), ('award', 'Xabier Lizardi award'), ('name', 'Leire Bilbao')]
[('year', '1982'), ('award', 'Xabier Lizardi award'), ('name', 'Alaine Agirre')]
[('year', '1982'), ('award', 'Xabier Lizardi award'), ('name', 'Nerea Arrien')]
[('year', '1982'), ('award', 'Xabier Lizardi award'), ('name', 'Ramon Olasagasti Aiestaran')]
[('year', '1982'), ('award', 'Xabier Lizardi award'), ('name', 'Oihane Fernandez')]
[('year', '1982'), ('award', 'Xabier Lizardi award'), ('name', 'Idoia Etxeberria')]
[('year', '1983'), ('award', 'Xabier Lizardi award'), ('name', 'minus sign')]

50

Now print the result. I need also to match the nation (with the predicate `P17` and `P27`) and filter over the year > 2000

In [31]:
queryString = """
SELECT DISTINCT ?country (COUNT(*) AS ?awards)
WHERE {  
?sub wdt:P279+ wd:Q618779 .
?sp wdt:P31 ?sub;
    wdt:P585 ?o;
    wdt:P1346 ?win.
?win wdt:P27|wdt:P17 ?c.

?c sc:name ?country .
BIND(year(xsd:dateTime(?o)) AS ?year)
FILTER (?year>2000)
} 
GROUP BY ?c ?country
ORDER BY DESC(?awards) 
LIMIT 50
"""

print("Results")
run_query(queryString)

Results
[('country', 'Spain'), ('awards', '166')]
[('country', 'United States of America'), ('awards', '19')]
[('country', 'Czech Republic'), ('awards', '17')]
[('country', 'France'), ('awards', '6')]
[('country', 'Chile'), ('awards', '6')]
[('country', 'Czechoslovakia'), ('awards', '6')]
[('country', 'United Kingdom'), ('awards', '6')]
[('country', 'Germany'), ('awards', '5')]
[('country', 'Finland'), ('awards', '3')]
[('country', 'India'), ('awards', '3')]
[('country', 'Romania'), ('awards', '2')]
[('country', 'United Kingdom of Great Britain and Ireland'), ('awards', '2')]
[('country', 'Liberia'), ('awards', '2')]
[('country', 'Bangladesh'), ('awards', '2')]
[('country', 'French'), ('awards', '2')]
[('country', 'State of Palestine'), ('awards', '2')]
[('country', 'Democratic Republic of the Congo'), ('awards', '1')]
[('country', 'Italy'), ('awards', '1')]
[('country', 'Pakistan'), ('awards', '1')]
[('country', 'Iraq'), ('awards', '1')]
[('country', 'Sweden'), ('awards', '1')]
[('cou

34

#### Considerations
The result shows that France won a lot of Nobel Prize compared to Italy and Iran. 
In the last 20 years spain won a lot of awards and in fact is the nation with more awards won.

## Task 3
Determine how many Literature nobel laureates have the following contries: Italy, Germany, France, Romania, Denmark, Iran, and China

In [32]:
#starting look at the country of interest
queryString = """
SELECT DISTINCT ?c ?country
WHERE{ 
    wd:Q38 wdt:P31 ?o . 
    ?c wdt:P31 ?o.
    ?c sc:name ?country .
    ?o sc:name ?obj .
    FILTER(REGEX(?country, "[I,i]taly") OR REGEX(?country, "[G,g]ermany") OR REGEX(?country, "[F,f]rance") 
    OR REGEX(?country, "[R,r]omania") OR REGEX(?country, "[D,d]enmark") OR REGEX(?country, "[I,i]ran") 
    OR REGEX(?country, "[C,c]hina") )
    
}
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('c', 'http://www.wikidata.org/entity/Q148'), ('country', "People's Republic of China")]
[('c', 'http://www.wikidata.org/entity/Q107312248'), ('country', 'Republic of China')]
[('c', 'http://www.wikidata.org/entity/Q794'), ('country', 'Iran')]
[('c', 'http://www.wikidata.org/entity/Q107258515'), ('country', 'Pahlavi Iran')]
[('c', 'http://www.wikidata.org/entity/Q35'), ('country', 'Denmark')]
[('c', 'http://www.wikidata.org/entity/Q218'), ('country', 'Romania')]
[('c', 'http://www.wikidata.org/entity/Q142'), ('country', 'France')]
[('c', 'http://www.wikidata.org/entity/Q183'), ('country', 'Germany')]
[('c', 'http://www.wikidata.org/entity/Q7318'), ('country', 'Nazi Germany')]
[('c', 'http://www.wikidata.org/entity/Q38'), ('country', 'Italy')]


10

Put the pattern above inside the next query to filter the country of interest

In [33]:
queryString = """
SELECT DISTINCT ?country (COUNT(*) AS ?count)
WHERE {   
?p wdt:P166 wd:Q37922;
    wdt:P27 ?c.
#part related to the country
wd:Q38 wdt:P31 ?o . 
?c wdt:P31 ?o.
?c sc:name ?country .
FILTER(REGEX(?country, "[I,i]taly") OR REGEX(?country, "[G,g]ermany") OR REGEX(?country, "[F,f]rance") 
    OR REGEX(?country, "[R,r]omania") OR REGEX(?country, "[D,d]enmark") OR REGEX(?country, "[I,i]ran") 
    OR REGEX(?country, "[C,c]hina") )    
}
GROUP BY ?c ?country
ORDER BY DESC(?count) 
"""

print("Results")
run_query(queryString)

Results
[('country', 'France'), ('count', '34')]
[('country', 'Denmark'), ('count', '8')]
[('country', 'Italy'), ('count', '6')]
[('country', 'Germany'), ('count', '6')]
[('country', "People's Republic of China"), ('count', '4')]
[('country', 'Romania'), ('count', '2')]
[('country', 'Nazi Germany'), ('count', '1')]


7

Since there are one people that has `wdt:P27` Nazi Germany I want to understand if there is a duplicate of a people that is also a German people

In [34]:
queryString = """
SELECT DISTINCT ?people ?country
WHERE {   
?p wdt:P166 wd:Q37922;
    wdt:P27 ?c.
#part related to the country
wd:Q38 wdt:P31 ?o . 
?c wdt:P31 ?o.
?c sc:name ?country .
?p sc:name ?people .
FILTER( REGEX(?country, "[G,g]ermany"))
}
"""

print("Results")
run_query(queryString)

Results
[('people', 'Nelly Sachs'), ('country', 'Germany')]
[('people', 'Hermann Hesse'), ('country', 'Germany')]
[('people', 'Heinrich Böll'), ('country', 'Germany')]
[('people', 'Günter Grass'), ('country', 'Germany')]
[('people', 'Herta Müller'), ('country', 'Germany')]
[('people', 'Rudolf Christoph Eucken'), ('country', 'Germany')]
[('people', 'Thomas Mann'), ('country', 'Nazi Germany')]


7

#### Considerations
France dominated this field on Nobel price since most of the winners are french. But Denmark is a very surprise!

## Task 4 
How many Literature Nobel award winners have a PhD (aka Doctorate of Philosophy) (you may check if they have a doctoral advisor)?

In [35]:
#look for a predicate similar to doctoral
queryString = """
SELECT DISTINCT ?pr ?name
WHERE {   
?p wdt:P166 wd:Q37922 ;
    ?pr ?o.

?pr sc:name ?name .
FILTER(REGEX(?name,"[d,D]octor"))
}
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P184'), ('name', 'doctoral advisor')]
[('pr', 'http://www.wikidata.org/prop/direct/P185'), ('name', 'doctoral student')]
[('pr', 'http://www.wikidata.org/prop/direct/P5459'), ('name', 'RHE doctor ID')]


3

Found `wdt:P184` that is 'doctoral advisor'. Now let see some object of these triple to understand what is.

In [36]:
#find who have an advisor
queryString = """
SELECT DISTINCT ?winner ?advisor
WHERE {   
?p wdt:P166 wd:Q37922 ;
    wdt:P184 ?o.

?o sc:name ?advisor .
?p sc:name ?winner .
}
"""

print("Results")
run_query(queryString)

Results
[('winner', 'Bertrand Russell'), ('advisor', 'Alfred North Whitehead')]
[('winner', 'Rudolf Christoph Eucken'), ('advisor', 'Friedrich Adolf Trendelenburg')]


2

Only two of the winners of a Literature Nobel Prize have a doctoral advisor. Let's print some numbers for comparisons

In [37]:
#find who have an advisor
queryString = """
SELECT ?phd ?total ?percentage
WHERE{
    {
        SELECT COUNT(DISTINCT ?p) AS ?phd
        WHERE {   
        ?p wdt:P166 wd:Q37922 ;
            wdt:P184 ?o.
        }
    }
    {
        SELECT COUNT(DISTINCT ?p) AS ?total 
        WHERE {   
        ?p wdt:P166 wd:Q37922.
        }
    }
    BIND( concat((xsd:float(?phd) / xsd:float(?total) * xsd:float(100) ),"%") AS ?percentage)
}
"""

print("Results")
run_query(queryString)

Results
[('phd', '2'), ('total', '118'), ('percentage', '1.694915294647217%')]


1

#### Considerations
I found that in our database there are only two people that have a doctoral advisor.

## Task 5
Are there books from Litarature Nobel Award winners which are not present in the Vatican Library? (if so, who is the author with more books not in the Vatical Library)?

In [38]:
#look for predicate related to book from a winner of a literature nobel prize
queryString = """
SELECT DISTINCT ?pr ?name
WHERE {   
?p wdt:P166 wd:Q37922 ;
    ?pr ?o.
?o wdt:P31 ?b.

?pr sc:name ?name .
?b sc:name ?book .
FILTER(REGEX(?book,"[B,b]ook"))
}
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P800'), ('name', 'notable work')]
[('pr', 'http://www.wikidata.org/prop/direct/P793'), ('name', 'significant event')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('pr', 'http://www.wikidata.org/prop/direct/P485'), ('name', 'archives at')]
[('pr', 'http://www.wikidata.org/prop/direct/P463'), ('name', 'member of')]
[('pr', 'http://www.wikidata.org/prop/direct/P136'), ('name', 'genre')]
[('pr', 'http://www.wikidata.org/prop/direct/P101'), ('name', 'field of work')]


7

In [39]:
#look for predicate related to book from a winner of a literature nobel prize
queryString = """
SELECT DISTINCT ?pr ?name
WHERE {   
?p wdt:P166 wd:Q37922 .
?o ?pr ?p.
?o wdt:P31 ?b.

?pr sc:name ?name .
?b sc:name ?book .
FILTER(REGEX(?book,"[B,b]ook"))
}
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P170'), ('name', 'creator')]
[('pr', 'http://www.wikidata.org/prop/direct/P50'), ('name', 'author')]
[('pr', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('pr', 'http://www.wikidata.org/prop/direct/P674'), ('name', 'characters')]
[('pr', 'http://www.wikidata.org/prop/direct/P921'), ('name', 'main subject')]
[('pr', 'http://www.wikidata.org/prop/direct/P2679'), ('name', 'author of foreword')]


6

In [40]:
#look for the object book
queryString = """
SELECT DISTINCT ?b ?book
WHERE {   
?p wdt:P166 wd:Q37922 .
?o ?pr ?p.
?o wdt:P31 ?b.

?pr sc:name ?name .
?b sc:name ?book .
FILTER(REGEX(?book,"[B,b]ook"))
}
"""

print("Results")
run_query(queryString)

Results
[('b', 'http://www.wikidata.org/entity/Q1062404'), ('book', "artist's book")]
[('b', 'http://www.wikidata.org/entity/Q254554'), ('book', 'picture book')]
[('b', 'http://www.wikidata.org/entity/Q277759'), ('book', 'book series')]
[('b', 'http://www.wikidata.org/entity/Q571'), ('book', 'book')]
[('b', 'http://www.wikidata.org/entity/Q128093'), ('book', 'e-book')]
[('b', 'http://www.wikidata.org/entity/Q53731850'), ('book', 'individual copy of a book')]
[('b', 'http://www.wikidata.org/entity/Q22759'), ('book', 'book censorship')]
[('b', 'http://www.wikidata.org/entity/Q2831984'), ('book', 'comic book album')]
[('b', 'http://www.wikidata.org/entity/Q20540385'), ('book', 'non-fiction book')]


9

Found the predicate author (`wdt:P50`) and the object book (`wd:Q571`). Let's print some (author,book)

In [41]:
#look for predicate related to book from a winner of a literature nobel prize
queryString = """
SELECT DISTINCT ?author ?book
WHERE {   
?p wdt:P166 wd:Q37922 .
?o wdt:P50 ?p.
?o wdt:P31/wdt:P279* wd:Q571.

?p sc:name ?author .
?o sc:name ?book .
}
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('author', 'Yasunari Kawabata'), ('book', 'Palm-of-the-Hand Stories')]
[('author', 'Olga Tokarczuk'), ('book', 'The Lost Soul')]
[('author', 'Knut Hamsun'), ('book', 'Hunger')]
[('author', 'Mario Vargas Llosa'), ('book', 'The Time of the Hero')]
[('author', 'Nadine Gordimer'), ('book', 'The Lying Days')]
[('author', 'Frédéric Mistral'), ('book', 'Lou Tresor dóu Felibrige')]
[('author', 'Olga Tokarczuk'), ('book', 'E.E.')]
[('author', 'Gabriel García Márquez'), ('book', 'Chronicle of a Death Foretold')]
[('author', 'Patrick Modiano'), ('book', 'Missing Person')]
[('author', 'Najib Mahfouz'), ('book', 'Arabian Nights and Days')]


10

Now let's find the relation from book and vatican library (`wd:Q213678`)

In [42]:
#look for predicate related to book from a winner of a literature nobel prize
queryString = """
SELECT DISTINCT ?pr ?name
WHERE {   
 wd:Q213678 ?pr ?p.

?pr sc:name ?name .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P5505'), ('name', 'WorldCat Registry ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1005'), ('name', 'Portuguese National Library ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1015'), ('name', 'NORAF ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1017'), ('name', 'Vatican Library ID (former scheme)')]
[('pr', 'http://www.wikidata.org/prop/direct/P1037'), ('name', 'director / manager')]
[('pr', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('pr', 'http://www.wikidata.org/prop/direct/P131'), ('name', 'located in the administrative territorial entity')]
[('pr', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('pr', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica On

60

In [43]:
#look for predicate related to book from a winner of a literature nobel prize
queryString = """
SELECT DISTINCT ?pr ?name
WHERE {   
?p ?pr wd:Q213678 .

?pr sc:name ?name .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('pr', 'http://www.wikidata.org/prop/direct/P2389'), ('name', 'organization directed by the office or position')]
[('pr', 'http://www.wikidata.org/prop/direct/P9419'), ('name', 'personal library at')]
[('pr', 'http://www.wikidata.org/prop/direct/P108'), ('name', 'employer')]
[('pr', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('pr', 'http://www.wikidata.org/prop/direct/P137'), ('name', 'operator')]
[('pr', 'http://www.wikidata.org/prop/direct/P1416'), ('name', 'affiliation')]
[('pr', 'http://www.wikidata.org/prop/direct/P189'), ('name', 'location of discovery')]
[('pr', 'http://www.wikidata.org/prop/direct/P195'), ('name', 'collection')]
[('pr', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('pr', 'http://www.wikidata.org/prop/direct/P301'), ('name', "category's main topic")]
[('pr', 'http://www.wikidata.org/prop/direct/P361'), ('name', 'part of')]
[('pr', 'http://www.wikidata.org/prop/direct/P485'), ('name', 'archives at')]
[('pr'

18

The property collection (`wdt:P195`) gives me all the book in the vatican library

In [44]:
#look for predicate related to book from a winner of a literature nobel prize
queryString = """
SELECT DISTINCT ?p ?name
WHERE {   
?p wdt:P195 wd:Q213678 .

?p sc:name ?name .
}
LIMIT 100
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/entity/Q17174857'), ('name', 'Bibbia di Federico da Montefeltro - BAV Urb.Lat.1&2')]
[('p', 'http://www.wikidata.org/entity/Q209285'), ('name', 'Codex Vaticanus')]
[('p', 'http://www.wikidata.org/entity/Q1806588'), ('name', 'Lasimos Krater')]
[('p', 'http://www.wikidata.org/entity/Q664820'), ('name', 'Bodmer Papyri')]
[('p', 'http://www.wikidata.org/entity/Q1106341'), ('name', 'Codex Vaticanus Graecus 64')]
[('p', 'http://www.wikidata.org/entity/Q3592714'), ('name', 'Barberini Gospels')]
[('p', 'http://www.wikidata.org/entity/Q16653651'), ('name', 'Divina Commedia de Federigo da Montefeltro - BAVaticana UrbLat365')]
[('p', 'http://www.wikidata.org/entity/Q903320'), ('name', 'Book of Dede Korkut')]
[('p', 'http://www.wikidata.org/entity/Q101438843'), ('name', 'Rossiana Library')]
[('p', 'http://www.wikidata.org/entity/Q1531846'), ('name', 'Vergilius Vaticanus')]
[('p', 'http://www.wikidata.org/entity/Q1937608'), ('name', 'Minuscule 131')]
[('p', '

55

In [45]:
#perform an ask query to answer
queryString = """
ASK
WHERE {   
?p wdt:P166 wd:Q37922 .
?o wdt:P50 ?p.
?o wdt:P31/wdt:P279* wd:Q571.
FILTER EXISTS{?o wdt:P195 wd:Q213678 .}
}
"""

print("Results")
run_ask_query(queryString)

Results


{'head': {'link': []}, 'boolean': False}

In [46]:
#find who write more books and there are not in the vatican library
queryString = """
SELECT ?author ( COUNT(DISTINCT ?o) AS ?books)
WHERE {   
?p wdt:P166 wd:Q37922 .
?o wdt:P50 ?p.
?o wdt:P31/wdt:P279* wd:Q571.
FILTER NOT EXISTS{?o wdt:P195 wd:Q213678 .}

?p sc:name ?author .
}
GROUP BY ?p ?author
ORDER BY DESC (?books)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('author', 'François Mauriac'), ('books', '15')]
[('author', 'Wole Soyinka'), ('books', '15')]
[('author', 'Yasunari Kawabata'), ('books', '11')]
[('author', 'Anatole France'), ('books', '9')]
[('author', 'André Gide'), ('books', '9')]
[('author', 'Romain Rolland'), ('books', '8')]
[('author', 'Henryk Sienkiewicz'), ('books', '6')]
[('author', 'Olga Tokarczuk'), ('books', '6')]
[('author', 'Shmuel Yosef Agnon'), ('books', '6')]
[('author', 'Toni Morrison'), ('books', '5')]


10

#### Consideration
There are no winner of Nobel Prize in literature that are authors of some books present in the Vatican library.
François Mauriac and Wole Soyinka are the two winners of Literature Nobel prize that have more books not in the Vatical Library.